<a href="https://colab.research.google.com/github/MineManiac/superComputacao-2024.2/blob/main/entregaAula12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile exercicio1.cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/random.h>
#include <thrust/host_vector.h>
#include <iostream>
#include <cstdlib>
#include <ctime>

// Estrutura saxpy
struct saxpy
{
    int a;
    saxpy(int a_) : a(a_) {};  // Construtor que inicializa a constante 'a'

    __host__ __device__
    double operator()(const int& x, const int& y) const {
        return a * x + y;  // Operação SAXPY
    }
};

// Função para gerar vetores aleatórios
void generate_random_vector(thrust::host_vector<int>& vec) {
    srand(static_cast<unsigned>(time(0)));  // Seed para números aleatórios
    for(size_t i = 0; i < vec.size(); i++) {
        vec[i] = rand() % 100;  // Gera valores aleatórios entre 0 e 99
    }
}

int main() {
    int n = 1000;  // Tamanho dos vetores
    int a = 5;     // Constante 'a' para SAXPY

    // Vetores na CPU (host)
    thrust::host_vector<int> h_x(n);
    thrust::host_vector<int> h_y(n);
    thrust::host_vector<double> h_result(n);

    // Gera valores aleatórios para os vetores x e y
    generate_random_vector(h_x);
    generate_random_vector(h_y);

    // Copia os vetores da CPU para a GPU
    thrust::device_vector<int> d_x = h_x;
    thrust::device_vector<int> d_y = h_y;
    thrust::device_vector<double> d_result(n);

    // Aplica o cálculo SAXPY usando a função transform
    thrust::transform(d_x.begin(), d_x.end(), d_y.begin(), d_result.begin(), saxpy(a));

    // Copia o resultado de volta para o host
    thrust::copy(d_result.begin(), d_result.end(), h_result.begin());

    // Exibe os primeiros 10 resultados
    std::cout << "Primeiros 10 resultados SAXPY (a * x + y):" << std::endl;
    for (int i = 0; i < 10; ++i) {
        std::cout << h_x[i] << " * " << a << " + " << h_y[i] << " = " << h_result[i] << std::endl;
    }

    return 0;
}


Writing exercicio1.cu


In [2]:
!nvcc -arch=sm_70 -std=c++14 exercicio1.cu -o exercicio1

In [5]:
!./exercicio1

Primeiros 10 resultados SAXPY (a * x + y):
74 * 5 + 74 = 444
86 * 5 + 86 = 516
93 * 5 + 93 = 558
69 * 5 + 69 = 414
22 * 5 + 22 = 132
85 * 5 + 85 = 510
13 * 5 + 13 = 78
54 * 5 + 54 = 324
98 * 5 + 98 = 588
82 * 5 + 82 = 492


In [11]:
%%writefile magnitude.cu
#include <thrust/host_vector.h>     // Para vetores no host (CPU)
#include <thrust/device_vector.h>   // Para vetores no device (GPU)
#include <thrust/transform_reduce.h>  // Para operações de transformação e redução
#include <thrust/functional.h>      // Para operações como thrust::plus
#include <cmath>                    // Para a função sqrt
#include <iostream>                 // Para entrada e saída
#include <chrono>                   // Para medir o tempo de execução

// Functor para elevar ao quadrado
struct square
{
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;  // Função de elevação ao quadrado
    }
};

// Função que calcula a magnitude usando Thrust na GPU
float magnitude_gpu(thrust::device_vector<float>& v) {
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);
}

int main() {
    int n = 1000000;  // Tamanho do vetor

    // Vetor aleatório no host (CPU)
    thrust::host_vector<float> h_v(n);  // Certifique-se de ter incluído a biblioteca correta

    // Preenche o vetor com valores aleatórios
    for (int i = 0; i < n; ++i) {
        h_v[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Copia o vetor da CPU para a GPU
    thrust::device_vector<float> d_v = h_v;

    // Medição do tempo de execução na GPU
    auto gpu_start = std::chrono::steady_clock::now();
    float magnitude_gpu_result = magnitude_gpu(d_v);
    auto gpu_end = std::chrono::steady_clock::now();

    // Calcula o tempo de execução na GPU
    std::chrono::duration<double> gpu_diff = gpu_end - gpu_start;
    std::cout << "Magnitude (GPU): " << magnitude_gpu_result << std::endl;
    std::cout << "Tempo de execução na GPU: " << gpu_diff.count() << " segundos" << std::endl;

    return 0;
}


Writing magnitude.cu


In [12]:
!nvcc -arch=sm_70 -std=c++14 magnitude.cu -o magnitudecu

In [15]:
!./magnitudecu

Magnitude (GPU): 577.328
Tempo de execução na GPU: 0.00070064 segundos


In [16]:
%%writefile magnitude.c
#include <iostream>
#include <vector>
#include <cmath>
#include <chrono>

// Função que calcula a magnitude de um vetor na CPU
float magnitude_cpu(const std::vector<float>& v) {
    float sum_of_squares = 0.0f;
    for (size_t i = 0; i < v.size(); ++i) {
        sum_of_squares += v[i] * v[i];
    }
    return std::sqrt(sum_of_squares);
}

int main() {
    int n = 1000000;  // Tamanho do vetor

    // Vetor aleatório na CPU
    std::vector<float> v(n);
    for (int i = 0; i < n; ++i) {
        v[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Medição do tempo de execução na CPU
    auto cpu_start = std::chrono::steady_clock::now();
    float magnitude_cpu_result = magnitude_cpu(v);
    auto cpu_end = std::chrono::steady_clock::now();

    // Calcula o tempo de execução na CPU
    std::chrono::duration<double> cpu_diff = cpu_end - cpu_start;
    std::cout << "Magnitude (CPU): " << magnitude_cpu_result << std::endl;
    std::cout << "Tempo de execução na CPU: " << cpu_diff.count() << " segundos" << std::endl;

    return 0;
}

Writing magnitude.c


In [17]:
!g++ magnitude.c -o  magnitudec

In [18]:
!./magnitudec

Magnitude (CPU): 577.228
Tempo de execução na CPU: 0.00947636 segundos


In [19]:
%%writefile exercicio3.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <cmath>
#include <iostream>
#include <chrono>

// Functor para calcular a diferença ao quadrado em relação à média
struct variance_op
{
    float mean;
    variance_op(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

// Função para calcular a variância com kernel fusion
float calculate_variance(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

int main() {
    int n = 1000000;  // Tamanho do vetor

    // Vetor no host (CPU)
    thrust::host_vector<float> h_vec(n);

    // Preencher o vetor com valores aleatórios
    for (int i = 0; i < n; i++) {
        h_vec[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Copiar o vetor para o device (GPU)
    thrust::device_vector<float> d_vec = h_vec;

    // Calcular a média
    float mean = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>()) / n;

    // Medir o tempo de execução para o cálculo da variância com kernel fusion
    auto start = std::chrono::steady_clock::now();
    float variance = calculate_variance(d_vec, mean);
    auto end = std::chrono::steady_clock::now();
    std::chrono::duration<double> elapsed = end - start;

    std::cout << "Variância: " << variance << std::endl;
    std::cout << "Tempo de execução (kernel fusion): " << elapsed.count() << " segundos" << std::endl;

    return 0;
}


Writing exercicio3.cu


In [20]:
!nvcc -arch=sm_70 -std=c++14 exercicio3.cu -o exercicio3

In [23]:
!./exercicio3

Variância: 0.0833013
Tempo de execução (kernel fusion): 0.000234982 segundos
